In [1]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [4]:
df = pd.read_csv('data/reddit_unencode_length.csv', index_col=0)
df.head()

,text,label,text_length
0,I recently went through a breakup and she said...,depression,543
1,"I do not know how to navigate these feelings, ...",depression,1510
2,"So I have been with my bf for 5 months , and h...",depression,625
3,I am so exhausted of this. Just when I think I...,SuicideWatch,1704
4,I have been severly bullied since i was 5 till...,depression,1006


In [6]:
df.loc[1]['text']

'I do not know how to navigate these feelings, not that its a new feeling by any stretch. I just do not understand how I can go on from being so pleasantly fine to suicidal and self loathing within in minutes and not be able to step out of it. I have been suffering from poor mental health for over 10 years and I am about to turn 26 next week and I feel so ashamed for how serious my negative thoughts are. Covid was a hard transition especially having moved to a new city a few months before everything started locking down, but now I am here and settled and I have a loving partner, a great connection to their family, a really fun job and I finished my degree. But when I stop and look around I feel like nothing will ever be good enough for me and I will never be good enough, I will always have skin problems and bad teeth and acne. I will never be the artist I used to be (I have a Bachelor in fine art) because I do not have the same drive or passionate that I used to have for it. I will nev

In [8]:
# \r and \n
df['Content_Parsed_1'] = df['text'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")

In [9]:
text = "Mr Greenspan\'s"
text

"Mr Greenspan's"

In [10]:
# " when quoting text
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')

In [11]:
# Lowercasing the text
df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()

In [12]:
punctuation_signs = list("?:!.,;")
df['Content_Parsed_3'] = df['Content_Parsed_2']

for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')

C:\Users\sathy\AppData\Local\Temp\ipykernel_19676\1781969415.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')


In [13]:
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")

In [14]:
# Downloading punkt and wordnet from NLTK
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

------------------------------------------------------------


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sathy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sathy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
# Saving the lemmatizer into an object
wordnet_lemmatizer = WordNetLemmatizer()

In [16]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):

    # Create an empty list containing lemmatized words
    lemmatized_list = []

    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))

    # Join the list
    lemmatized_text = " ".join(lemmatized_list)

    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [18]:
df['Content_Parsed_5'] = lemmatized_text_list
df.head()

,text,label,text_length,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4,Content_Parsed_5
0,I recently went through a breakup and she said...,depression,543,I recently went through a breakup and she said...,i recently went through a breakup and she said...,i recently went through a breakup and she said...,i recently went through a breakup and she said...,i recently go through a breakup and she say sh...
1,"I do not know how to navigate these feelings, ...",depression,1510,"I do not know how to navigate these feelings, ...","i do not know how to navigate these feelings, ...",i do not know how to navigate these feelings n...,i do not know how to navigate these feelings n...,i do not know how to navigate these feel not t...
2,"So I have been with my bf for 5 months , and h...",depression,625,"So I have been with my bf for 5 months , and h...","so i have been with my bf for 5 months , and h...",so i have been with my bf for 5 months and he...,so i have been with my bf for 5 months and he...,so i have be with my bf for 5 months and he a...
3,I am so exhausted of this. Just when I think I...,SuicideWatch,1704,I am so exhausted of this. Just when I think I...,i am so exhausted of this. just when i think i...,i am so exhausted of this just when i think i ...,i am so exhausted of this just when i think i ...,i be so exhaust of this just when i think i ca...
4,I have been severly bullied since i was 5 till...,depression,1006,I have been severly bullied since i was 5 till...,i have been severly bullied since i was 5 till...,i have been severly bullied since i was 5 till...,i have been severly bullied since i was 5 till...,i have be severly bully since i be 5 till 15 t...


In [19]:
# Downloading the stop words list
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sathy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
# Loading the stop words in english
stop_words = list(stopwords.words('english'))

In [21]:
stop_words[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [22]:
example = "me eating a meal"
word = "me"

# The regular expression is:
regex = r"\b" + word + r"\b"  # we need to build it like that to work properly

re.sub(regex, "StopWord", example)

'StopWord eating a meal'

In [23]:
df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')

C:\Users\sathy\AppData\Local\Temp\ipykernel_19676\3814005232.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')


In [29]:
df.to_csv('data/reddit_unencode_length_lemmatized_allsteps.csv')

In [30]:
df.head()

,text,label,Content_Parsed
0,I recently went through a breakup and she said...,depression,recently go breakup say still want fri...
1,"I do not know how to navigate these feelings, ...",depression,know navigate feel new feel stretc...
2,"So I have been with my bf for 5 months , and h...",depression,bf 5 months already tell depress ...
3,I am so exhausted of this. Just when I think I...,SuicideWatch,exhaust think finally rest think ...
4,I have been severly bullied since i was 5 till...,depression,severly bully since 5 till 15 result ...


In [32]:
list_columns = ["text", "label", "Content_Parsed_6"]
df = df[list_columns]
df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})
df.head()

,text,label,Content_Parsed
0,I recently went through a breakup and she said...,depression,recently go breakup say still want fri...
1,"I do not know how to navigate these feelings, ...",depression,know navigate feel new feel stretc...
2,"So I have been with my bf for 5 months , and h...",depression,bf 5 months already tell depress ...
3,I am so exhausted of this. Just when I think I...,SuicideWatch,exhaust think finally rest think ...
4,I have been severly bullied since i was 5 till...,depression,severly bully since 5 till 15 result ...


In [33]:
category_codes = {
    'depression': 0,
    'SuicideWatch': 1
}

In [35]:
# Category mapping
df['Category_Code'] = df['label']
df = df.replace({'Category_Code':category_codes})
df.head()

,text,label,Content_Parsed,Category_Code
0,I recently went through a breakup and she said...,depression,recently go breakup say still want fri...,0
1,"I do not know how to navigate these feelings, ...",depression,know navigate feel new feel stretc...,0
2,"So I have been with my bf for 5 months , and h...",depression,bf 5 months already tell depress ...,0
3,I am so exhausted of this. Just when I think I...,SuicideWatch,exhaust think finally rest think ...,1
4,I have been severly bullied since i was 5 till...,depression,severly bully since 5 till 15 result ...,0


In [38]:
X_train, X_test, y_train, y_test = train_test_split(df['Content_Parsed'],
                                                    df['Category_Code'],
                                                    test_size=0.15,
                                                    random_state=8)

In [39]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

In [41]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)

features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(17308, 300)
(3055, 300)


In [42]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(category_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")

# 'SuicideWatch' category:
  . Most correlated unigrams:
. die
. suicide
. depress
. kill
. depression
  . Most correlated bigrams:
. feel like
. want die

# 'depression' category:
  . Most correlated unigrams:
. die
. suicide
. depress
. kill
. depression
  . Most correlated bigrams:
. feel like
. want die



C:\Users\sathy\OneDrive\Documents\GitHub\humm\venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\sathy\OneDrive\Documents\GitHub\humm\venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [43]:
bigrams

['cannot even',
 'every day',
 'get better',
 'even though',
 'mental health',
 'make feel',
 'feel like',
 'want die']

In [45]:
# X_train
with open('Pickles/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)

# X_test
with open('Pickles/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)

# y_train
with open('Pickles/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)

# y_test
with open('Pickles/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)

# df
with open('Pickles/df.pickle', 'wb') as output:
    pickle.dump(df, output)

# features_train
with open('Pickles/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('Pickles/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open('Pickles/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('Pickles/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)

# TF-IDF object
with open('Pickles/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)